In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# read csv to dataframes

In [5]:
departments=pd.read_csv('departments.csv')
aisles=pd.read_csv('aisles.csv')
products=pd.read_csv('products.csv')
# sample_submission=pd.read_csv('/home/user_ds/qianpeng/data/data/sample_submission.csv')
order_products_train=pd.read_csv('order_products__train.csv')
order_products_prior=pd.read_csv('order_products__prior.csv')
orders=pd.read_csv('orders.csv')
orders_prior=orders[orders.eval_set=='prior']
orders_train=orders[orders.eval_set=='train']

In [8]:
order_products_train

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [10]:
orders_train

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0
...,...,...,...,...,...,...,...
3420838,2585586,206199,train,20,2,16,30.0
3420862,943915,206200,train,24,6,19,6.0
3420924,2371631,206203,train,6,4,19,30.0
3420933,1716008,206205,train,4,1,16,10.0


In [207]:
#  ------ in orders ------
# test - 75,000 - 36.4%
# prior - 3,214,874 orders # 32,434,489
# train - 131,209 - 63.6%
# total user - 206,209

# ------- in order_products ------
# order_products_prior - 32,434,489 , 13,307,953 (groupby user and products )
# order_products_train - 1,384,617
# 13,307,953
# each user has 15.6 orders on average, max 99, min 3, median 9, 

15.590367054784224

# Merging to get order_master df

## Merging products, aisles and departments df together




In [65]:
# replace space in prodt name with -
products.product_name = products.product_name.str.replace(' ', '-')

products_details = pd.merge(left=pd.merge(left=products, right=departments, how='left'), right=aisles, how='left')

## use prior set only to create order master df 

In [66]:
order_master = order_products_prior.copy()
order_master = pd.merge(order_master, products_details, on='product_id', how='left')
order_master = pd.merge(order_master, orders, on='order_id', how='left')


### create a new cln for reverse order number

In [5]:
order_master['order_number_reverse'] = order_master.groupby('user_id').order_number.transform(np.max) - order_master.order_number

### view user 1's order history

In [87]:

order_master.loc[order_master['user_id'] == 1].sort_values(['order_id'],ascending=False)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
31927073,3367565,25133,4,1,Organic-String-Cheese,21,16,dairy eggs,packaged cheese,1,prior,6,2,7,19.0
31927072,3367565,10258,3,1,Pistachios,117,19,snacks,nuts seeds dried fruit,1,prior,6,2,7,19.0
31927071,3367565,12427,2,1,Original-Beef-Jerky,23,19,snacks,popcorn jerky,1,prior,6,2,7,19.0
31927070,3367565,196,1,1,Soda,77,7,beverages,soft drinks,1,prior,6,2,7,19.0
29474810,3108588,49235,6,0,Organic-Half-&-Half,53,16,dairy eggs,cream,1,prior,8,1,14,14.0
29474809,3108588,46149,5,0,Zero-Calorie-Cola,77,7,beverages,soft drinks,1,prior,8,1,14,14.0
29474808,3108588,25133,4,1,Organic-String-Cheese,21,16,dairy eggs,packaged cheese,1,prior,8,1,14,14.0
29474807,3108588,10258,3,1,Pistachios,117,19,snacks,nuts seeds dried fruit,1,prior,8,1,14,14.0
29474806,3108588,196,2,1,Soda,77,7,beverages,soft drinks,1,prior,8,1,14,14.0
29474805,3108588,12427,1,1,Original-Beef-Jerky,23,19,snacks,popcorn jerky,1,prior,8,1,14,14.0


# Split into Non Validation and Validation Sets

In [ ]:
# if it is the last order in the prior set, then it is in the validation data set. Otherwise in the non-validation set
order_master['is_validation_data'] = np.where(order_master['order_number_reverse']==0 , True, False)

validation_data = order_master[order_master['is_validation_data'] == True]
non_validation_data = order_master[order_master['is_validation_data'] == False]
